In [2]:
!pip install pytorch_lightning
!pip install wandb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 801.6/801.6 kB 15.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 840.4/840.4 kB 51.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 27.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 195.4/195.4 kB 23.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.5/258.5 kB 30.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 8.0 MB/s eta 0:00:00


In [3]:
import torch
import torch.optim as optim
import torch.nn.functional as F
import torch.nn as nn
import torchvision
from torch.utils.data import DataLoader, random_split
import pytorch_lightning as pl
import wandb

class MNISTDataModule(pl.LightningDataModule):
    def __init__(self, batch_size=128, data_dir: str = "~/torch_datasets"):
        super().__init__()
        self.data_dir = data_dir
        self.batch_size = batch_size
        self.transform = torchvision.transforms.Compose([
            torchvision.transforms.ToTensor(),
            torchvision.transforms.Normalize((0.5,), (0.5,))
        ])
        self.dims = (1, 28, 28)
        self.num_classes = 10

    def prepare_data(self):
        torchvision.datasets.MNIST(self.data_dir, train=True, download=True)
        torchvision.datasets.MNIST(self.data_dir, train=False, download=True)

    def setup(self, stage=None):
        if stage == 'fit' or stage is None:
            mnist_full = torchvision.datasets.MNIST(self.data_dir, train=True, transform=self.transform)
            self.train_dataset, self.val_dataset = random_split(mnist_full, [55000, 5000])
        if stage == 'test' or stage is None:
            self.test_dataset = torchvision.datasets.MNIST(self.data_dir, train=False, transform=self.transform)

    def train_dataloader(self):
        return DataLoader(self.train_dataset, batch_size=self.batch_size, shuffle=True)

    def val_dataloader(self):
        return DataLoader(self.val_dataset, batch_size=self.batch_size)

    def test_dataloader(self):
        return DataLoader(self.test_dataset, batch_size=self.batch_size)

class AE(pl.LightningModule):
    def __init__(self, input_shape=784, learning_rate=1e-3, epochs=10, batch_size=128):
        super().__init__()
        self.save_hyperparameters()  # Save hyperparameters for logging
        self.encoder_hidden_layer = nn.Linear(in_features=input_shape, out_features=128)
        self.encoder_output_layer = nn.Linear(in_features=128, out_features=128)
        self.decoder_hidden_layer = nn.Linear(in_features=128, out_features=128)
        self.decoder_output_layer = nn.Linear(in_features=128, out_features=input_shape)

    def forward(self, features):
        activation = self.encoder_hidden_layer(features)
        activation = torch.relu(activation)
        code = self.encoder_output_layer(activation)
        code = torch.relu(code)
        activation = self.decoder_hidden_layer(code)
        activation = torch.relu(activation)
        activation = self.decoder_output_layer(activation)
        reconstructed = torch.relu(activation)
        return reconstructed

    def training_step(self, batch, batch_idx):
        x, _ = batch
        reconstructions = self(x.view(x.size(0), -1))
        loss = F.mse_loss(reconstructions, x.view(x.size(0), -1))
        self.log('train_loss', loss, on_step=True, on_epoch=True, prog_bar=True, logger=True)

        # Log images
        if batch_idx % 100 == 0:
            original = x[:8]  # Log the first 8 images
            reconstructed = reconstructions[:8]  # Log the first 8 reconstructed images
            self.log_images(original, reconstructed, prefix='train')

        return loss

    def validation_step(self, batch, batch_idx):
        x, _ = batch
        reconstructions = self(x.view(x.size(0), -1))
        loss = F.mse_loss(reconstructions, x.view(x.size(0), -1))
        self.log('val_loss', loss, on_step=False, on_epoch=True, prog_bar=True, logger=True)

        # Log images
        if batch_idx % 100 == 0:
            original = x[:8]  # Log the first 8 images
            reconstructed = reconstructions[:8]  # Log the first 8 reconstructed images
            self.log_images(original, reconstructed, prefix='val')

        return loss

    def test_step(self, batch, batch_idx):
        x, _ = batch
        reconstructions = self(x.view(x.size(0), -1))
        loss = F.mse_loss(reconstructions, x.view(x.size(0), -1))
        self.log('test_loss', loss, on_step=False, on_epoch=True, prog_bar=True, logger=True)

        # Log images
        if batch_idx % 100 == 0:
            original = x[:8]  # Log the first 8 images
            reconstructed = reconstructions[:8]  # Log the first 8 reconstructed images
            self.log_images(original, reconstructed, prefix='test')

        return loss

    def log_images(self, original, reconstructed, prefix='train'):
        original_grid = torchvision.utils.make_grid(original, nrow=8, normalize=True)
        reconstructed_grid = torchvision.utils.make_grid(reconstructed, nrow=8, normalize=True)
        wandb.log({f'{prefix}_original': [wandb.Image(original_grid)]})
        wandb.log({f'{prefix}_reconstructed': [wandb.Image(reconstructed_grid)]})

    def configure_optimizers(self):
        optimizer = optim.Adam(self.parameters(), lr=self.hparams.learning_rate)
        return optimizer

# Initialize Wandb
wandb.init(project='autoencoder')
"""
run = wandb.init()
artifact = run.use_artifact('oudaisalameh/autoencoder/run-u2ri2izv-history:v0', type='wandb-history')
artifact_dir = artifact.download()
"""

"""
from my_module import MyModel  # Import your LightningModule subclass
checkpoint = torch.load("checkpoint.ckpt")
# Load the model from the checkpoint
model = MyModel.load_from_checkpoint(checkpoint_path)
"""

# Create AE model
model = AE()

# DataModule
dm = MNISTDataModule()
dm.prepare_data()
dm.setup()

# Callbacks
checkpoint_callback = pl.callbacks.ModelCheckpoint(
    monitor='val_loss',
    dirpath='./checkpoints',
    filename='autoencoder-{epoch:02d}-{val_loss:.2f}',
    save_top_k=1,
    mode='min',
    save_weights_only=False  # Save entire model
)

# Trainer
trainer = pl.Trainer(max_epochs=model.hparams.epochs, logger=pl.loggers.WandbLogger(), callbacks=[checkpoint_callback])#,resume_from_checkpoint='checkpoint.ckpt'

# Training
trainer.fit(model, dm)

# Testing
trainer.test(datamodule=dm)

wandb.finish()  # Close the wandb run after training and testing finishes


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


100%|██████████| 9912422/9912422 [00:00<00:00, 117038851.57it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=10` reached.
/usr/local/lib/python3.10/dist-packages/pytorch_lightning/trainer/connectors/checkpoint_connector.py:145: `.test(ckpt_path=None)` was called without a model. The best model of the previous `fit` call will be used. You can pass `.test(ckpt_path='best')` to use the best model or `.test(ckpt_path='last')` to use the last model. If you pass a value, this warning will be silenced.
INFO:pytorch_lightning.utilities.rank_zero:Restoring states from the checkpoint path at /content/checkpoints/autoencoder-epoch=09-val_loss=0.89.ckpt
INFO:pytorch_lightning.utilities.rank_zero:Loaded model weights from the checkpoint at /content/checkpoints/autoencoder-epoch=09-val_loss=0.89.ckpt


Testing: |          | 0/? [00:00<?, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_loss         │    0.8901015520095825     │
└───────────────────────────┴───────────────────────────┘

epoch,▁▁▁▁▂▂▂▂▂▂▂▂▃▃▃▃▄▄▄▄▅▅▅▅▅▅▅▅▆▆▆▇▇▇▇▇▇▇▇█
test_loss,▁
train_loss_epoch,█▃▂▂▂▂▁▁▁▁
train_loss_step,█▆▅▄▃▄▃▂▃▂▃▃▂▂▃▃▃▃▃▂▂▂▃▂▂▂▁▂▂▁▂▂▂▃▂▂▂▂▃▂
trainer/global_step,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇████
val_loss,█▄▃▂▂▂▁▁▁▁
epoch,10
test_loss,0.8901
train_loss_epoch,0.89093
train_loss_step,0.8895
trainer/global_step,4300
